In [1]:
from pathlib import Path

import torch
import torch.nn as nn
import torch.utils.data as data
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
import torchvision.models as models
import net

def train_transform():
    transform_list = [
        transforms.Resize(size=(256,256)),
        transforms.ToTensor()
    ]
    return transforms.Compose(transform_list)


class FlatFolderDataset(data.Dataset):
    def __init__(self, root, transform):
        super(FlatFolderDataset, self).__init__()
        self.root = root
        self.paths = list(Path(self.root).glob('*'))
        self.transform = transform

    def __getitem__(self, index):
        path = self.paths[index]
        img = Image.open(str(path)).convert('RGB')
        img = self.transform(img)
        return img

    def __len__(self):
        return len(self.paths)

    def name(self):
        return 'FlatFolderDataset'

decoder = net.decoder
#decoder加载decoder.pth
decoder.load_state_dict(torch.load("decoder.pth"))
vgg=models.vgg16(pretrained=True).features
vgg = nn.Sequential(*list(vgg.children())[:31])
network = net.Net(vgg, decoder)
network.train()
print(network)



Net(
  (enc_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (enc_2): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
  )
  (enc_3): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): ReLU(inplace=True)
    (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
  )
  (enc_4): Sequen

In [6]:
tf=train_transform()
content_dataset = FlatFolderDataset(f"D:\git\AdaIN\content\images", tf)
style_dataset = FlatFolderDataset(f"D:\git\AdaIN\style\images", tf)

content_iter = iter(data.DataLoader(
    content_dataset, batch_size=2))
style_iter = iter(data.DataLoader(
    style_dataset, batch_size=2))

In [8]:
from torchvision.utils import save_image
network.to('cuda')
optimizer = torch.optim.Adam(network.parameters())
for i in tqdm(range(100000)):
    content_images = next(content_iter).to('cuda')
    style_images = next(style_iter).to('cuda')
    loss_c, loss_s, generate_image = network(content_images, style_images)
    loss_c = 1 * loss_c
    loss_s = 20 * loss_s
    loss = loss_c + loss_s

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (i+1) % 10 == 0:
        print(f"loss_c: {loss_c}, loss_s: {loss_s}")
    if (i+1) % 100 ==0:
        save_image(generate_image, f"output/generate_image_{i+1}.png")
        save_image(content_images, f"output/content_image_{i+1}.png")
        save_image(style_images, f"output/style_image_{i+1}.png")
    if (i+1) % 10000 == 0:
        torch.save(network.state_dict(), f"output/network_{i+1}.pth")

  0%|          | 12/100000 [00:01<4:20:36,  6.39it/s]

loss_c: 1.0574362622245136e-17, loss_s: 0.0005502101848833263


  0%|          | 20/100000 [00:03<5:45:25,  4.82it/s]

loss_c: 3.294958991999841e-18, loss_s: 0.0002924492582678795


  0%|          | 32/100000 [00:05<4:19:58,  6.41it/s]

...

loss_c: 6.268043809681823e-20, loss_s: 2.0903611584799364e-05


  0%|          | 342/100000 [00:58<4:23:48,  6.30it/s]

loss_c: 2.3801632021700434e-18, loss_s: 2.699739980016602e-06


  0%|          | 348/100000 [00:59<4:44:19,  5.84it/s]


KeyboardInterrupt: 

  5%|▌         | 51/1000 [00:14<03:50,  4.12it/s]

第50次迭代，content损失为：0.000，style损失为：18.842


 10%|█         | 101/1000 [00:26<03:39,  4.09it/s]

第100次迭代，content损失为：0.000，style损失为：5.658


 15%|█▌        | 151/1000 [00:39<03:29,  4.04it/s]

第150次迭代，content损失为：0.000，style损失为：1.878


 20%|██        | 200/1000 [00:51<04:12,  3.17it/s]

第200次迭代，content损失为：0.000，style损失为：0.458


 21%|██        | 208/1000 [00:53<03:23,  3.89it/s]


KeyboardInterrupt: 

Net(
  (enc_1): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): MaxP